In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
lying_df=pd.read_csv('눕기.csv',engine='python')

In [3]:
working_df=pd.read_csv('걷기.csv',engine='python')

In [4]:
standing_df=pd.read_csv('서기.csv',engine='python')

In [5]:
sitting_df=pd.read_csv('앉기.csv',engine='python')

In [6]:
facedown_df=pd.read_csv('엎드려눕기.csv',engine='python')

In [7]:
r_lying_df=pd.read_csv('오른쪽눕기.csv',engine='python')

In [8]:
l_lying_df=pd.read_csv('왼쪽눕기.csv',engine='python')

In [9]:
lying_df=pd.concat([lying_df,facedown_df,r_lying_df,l_lying_df],axis=0)

In [10]:
lying_df.shape

(20472, 6)

In [11]:
def processing_df(df):
    tmp_df=df.drop(['epoch (ms)','time (18:00)'],axis=1)
    tmp_df['elapsed (s)']=round(tmp_df['elapsed (s)'],0)
    tmp_df=tmp_df.groupby('elapsed (s)').mean()
    tmp_df=tmp_df.reset_index()
    
    proc_df=tmp_df.drop(['elapsed (s)'],axis=1)
    proc_df['elapsed (s)']=tmp_df['elapsed (s)']
    return proc_df

In [12]:
def add_action(df, action):
    tmp_df=processing_df(df)
    tmp_df['action']=[action]*tmp_df.shape[0]
    return tmp_df

In [13]:
action_df=pd.concat([add_action(lying_df,'lying'),add_action(standing_df,'standing'),add_action(sitting_df,'sitting')],axis=0)

action_df=pd.concat([add_action(lying_df,'lying'),add_action(standing_df,'standing')],axis=0)


In [14]:
action_df.head()

,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),elapsed (s),action
0,-1.311967,-0.005083,0.219617,0.0,lying
1,-1.321135,-0.038135,0.374865,1.0,lying
2,-1.076688,0.103573,0.283385,2.0,lying
3,-1.241740,-0.063542,0.351385,3.0,lying
4,-1.291759,-0.001139,0.339435,4.0,lying


In [502]:
from sklearn.model_selection import train_test_split

In [503]:
x=action_df.drop(['action'],axis=1)

In [504]:
x.head()

,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),elapsed (s)
0,-1.311967,-0.005083,0.219617,0.0
1,-1.321135,-0.038135,0.374865,1.0
2,-1.076688,0.103573,0.283385,2.0
3,-1.241740,-0.063542,0.351385,3.0
4,-1.291759,-0.001139,0.339435,4.0


In [505]:
x['mean']=x.mean(axis=1)
x['var']=x.var(axis=1)
x['max']=x.max(axis=1)
x['min']=x.min(axis=1)
x['kurtosis']=x.kurtosis(axis=1)
x['skewness']=x.skew(axis=1)

In [506]:
x.head()

,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),elapsed (s),mean,var,max,min,kurtosis,skewness
0,-1.311967,-0.005083,0.219617,0.0,-0.274358,0.367106,0.367106,-1.311967,-0.532320,-0.884609
1,-1.321135,-0.038135,0.374865,1.0,0.003898,0.721829,1.000000,-1.321135,-0.903406,-0.357823
2,-1.076688,0.103573,0.283385,2.0,0.327568,1.205272,2.000000,-1.076688,-1.229082,0.196073
3,-1.241740,-0.063542,0.351385,3.0,0.511526,2.405698,3.000000,-1.241740,-1.740923,0.295851
4,-1.291759,-0.001139,0.339435,4.0,0.761634,3.865878,4.000000,-1.291759,-1.987493,0.393785


In [779]:
X_train,X_test,y_train,y_test=train_test_split(x,action_df.action,stratify=action_df.action,random_state=1)

## SVC

In [780]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [781]:
std_scaler=StandardScaler()
std_scaler.fit(X_train)
std_scaler.fit(X_train)
std_X_train=std_scaler.transform(X_train)
std_X_test=std_scaler.transform(X_test)

In [782]:
svc=SVC()

In [783]:
svc.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [784]:
svc.score(X_test,y_test)

0.20382165605095542

### standard preproc

In [785]:
svc.fit(std_X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [786]:
svc.score(std_X_test,y_test)

0.40127388535031849

## MLPClassifier (solver = lbfgs)

In [787]:
from sklearn.neural_network import MLPClassifier

In [788]:
mlp=MLPClassifier(solver='lbfgs',random_state=0).fit(X_train, y_train)

In [789]:
mlp.score(X_test,y_test)

0.28025477707006369

### standard preproc

In [823]:
from sklearn.pipeline import Pipeline

In [824]:
pipe=Pipeline([('scaler',StandardScaler()),('mlp',MLPClassifier(solver='lbfgs',hidden_layer_sizes=(100,),random_state=0))])

In [825]:
pipe.fit(std_X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rat...      solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))])

In [826]:
pipe.score(std_X_test,y_test)

0.5859872611464968